# Домашняя работа 1


In [ ]:
# !pip install torch torchvision torchaudio torch-geometric ogb --quiet


In [ ]:
from ogb.nodeproppred import NodePropPredDataset
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import heapq


1. Загрузка данных

In [ ]:
dataset = NodePropPredDataset(name='ogbn-arxiv', root='data')
graph, labels = dataset[0]

print("Количество узлов:", graph["num_nodes"])
print("Количество рёбер:", graph["edge_index"].shape[1])


In [ ]:
edge_index = graph["edge_index"].T  # np.array (1.1M × 2)
G = nx.DiGraph()
G.add_edges_from(edge_index)
G = G.to_undirected()

Визуализируйте граф на семпле из 1000 узлов

In [ ]:
# Визуализация графа на семпле из 1000 узлов
sample_nodes = list(G.nodes())[:1000]
G_sample = G.subgraph(sample_nodes)

plt.figure(figsize=(15, 15))
pos = nx.spring_layout(G_sample, k=0.5, iterations=50)
nx.draw_networkx(G_sample, pos, 
                 node_size=30, 
                 node_color='lightblue',
                 edge_color='gray',
                 with_labels=False,
                 alpha=0.7,
                 width=0.3)
plt.title('Визуализация графа (1000 узлов)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

### Предварительный анализ

Произведем предварительный анализ графа

In [ ]:
# Средняя степень
degrees = [d for n, d in G.degree()]
avg_degree = np.mean(degrees)

# Плотность графа
density = nx.density(G)

# Для диаметра и радиуса используем наибольшую связную компоненту
# так как граф может быть несвязным
largest_cc = max(nx.connected_components(G), key=len)
G_lcc = G.subgraph(largest_cc)

# Диаметр графа (для наибольшей связной компоненты)
diameter = nx.diameter(G_lcc)

# Радиус графа (для наибольшей связной компоненты)
radius = nx.radius(G_lcc)

print(f"Средняя степень: {avg_degree:.2f}")
print(f"Плотность: {density:.6f}")
print(f"Диаметер графа: {diameter}")
print(f"Радиус графа: {radius}")

### Анализ на центральность

Найдите узел с максимальной центральностью

In [ ]:
# Вычисление центральности по степени (degree centrality)
degree_centrality_dict = nx.degree_centrality(G)
degree_centrality = max(degree_centrality_dict.items(), key=lambda x: x[1])
print("Центральность по степени (узел, значение):", degree_centrality)

# Вычисление центральности по близости (closeness centrality)
# Для большого графа вычисляем на подвыборке
closeness_centrality_dict = nx.closeness_centrality(G_sample)
closeness_centrality = max(closeness_centrality_dict.items(), key=lambda x: x[1])
print("Центральность по близости (узел, значение) [на выборке]:", closeness_centrality)

# Вычисление центральности по междуузловой значимости (betweenness centrality)
# Для большого графа используем аппроксимацию
betweenness_centrality_dict = nx.betweenness_centrality(G_sample, k=100)
betweenness_centrality = max(betweenness_centrality_dict.items(), key=lambda x: x[1])
print("Центральность по междуузловой значимости (узел, значение) [на выборке]:", betweenness_centrality)


Анализ кластеров

In [ ]:
# Вычислите средний кластерный коэффициент графа
global_cluster_coeff = nx.average_clustering(G)
print("Cредний кластерный коэффициент графа:", global_cluster_coeff)


In [ ]:
# Вычислите коэффициент транзитивности
transitivity_coeff = nx.transitivity(G)
print("Коэффициент транзитивности графа:", transitivity_coeff)


### Поиск сообществ

Реализуйте 2 метода поиска сообществ:
1. С помощью готовой реализации в библиотеке NetworkX
2. Реализуйте самостоятельно класс с алгоритмом Clauset–Newman–Moore

Сравните полученные результаты

Алгоритм Clauset–Newman–Moore (CNM)

---

**1. Начальное состояние**
- Каждая вершина — отдельное сообщество.  
- Вычисляем степени вершин и матрицу смежности.

---

**2. Вычисляем для всех пар сообществ** $(r, s)$, между которыми есть хотя бы одно ребро:

$$
\Delta Q_{rs} = 2 \, (e_{rs} - a_r a_s)
$$

где:  

- $e_{rs}$ — доля рёбер между сообществами *r* и *s*, делённая на $2m$;  
- $a_r = \sum_t e_{rt}$ — доля концов рёбер, инцидентных *r*.

---

**2.1. Доля концов рёбер, инцидентных вершинам из $r$**

Это часть всех концов рёбер в графе, которые принадлежат вершинам, входящим в сообщество $r$.

Эта величина обозначается как $a_r$ и вычисляется по формуле:

$$
a_r = \frac{1}{2m} \sum_{i \in r} k_i
$$

где:  

- $k_i$ — степень вершины $i$ (сколько у неё рёбер);  
- $m$ — общее число рёбер в графе;  
- $2m$ — общее количество **концов рёбер** (так как каждое ребро имеет два конца).

---

**3. Выбираем пару $(r, s)$** с максимальным $\Delta Q$ и объединяем их.

---

**4. Обновляем:**
- значения $e_{rt}, a_r$;  
- новую модульность:  
  $$
  Q \leftarrow Q + \Delta Q
  $$

---

**5. Повторяем**, пока $\Delta Q > 0$.

---

**6. Выбираем состояние с максимальным $Q$** как оптимальное разбиение.


Описание алгоритма - https://arxiv.org/pdf/cond-mat/0408187

In [ ]:
class CNM:
    """
    Алгоритм Clauset–Newman–Moore для обнаружения сообществ.
    """

    def __init__(self, edges):
        self.graph = defaultdict(dict)
        for u, v, w in edges:
            if u == v: continue
            self.graph[u][v] = self.graph[u].get(v, 0) + w
            self.graph[v][u] = self.graph[v].get(u, 0) + w
        self.nodes = list(self.graph.keys())
        self.m = sum(sum(self.graph[u].values()) for u in self.nodes) / 2
        self.deg = {u: sum(self.graph[u].values()) for u in self.nodes}

    def fit(self, verbose=False):
        communities = {u: {u} for u in self.nodes}
        comm_id = {u: u for u in self.nodes}
        m2 = 2 * self.m
        a = {u: self.deg[u] / m2 for u in self.nodes}

        # Доля рёбер между сообществами
        e = defaultdict(dict)
        for u in self.nodes:
            for v, w in self.graph[u].items():
                cu, cv = comm_id[u], comm_id[v]
                if cu != cv:
                    e[cu][cv] = e[cu].get(cv, 0) + w / m2

        # Начальная модульность
        Q = -sum(a[c] ** 2 for c in a)

        # Основной цикл
        while True:
            best_dQ = 0
            best_pair = None

            # Найти пару (r, s) с максимальным ΔQ
            for r in list(e.keys()):
                for s in list(e[r].keys()):
                    if r >= s:
                        continue
                    dQ = 2 * (e[r].get(s, 0) - a[r] * a[s])
                    if dQ > best_dQ:
                        best_dQ = dQ
                        best_pair = (r, s)

            # Условие выхода
            if best_dQ <= 0 or best_pair is None:
                break

            r, s = best_pair
            if verbose:
                print(f"Объединяем {r}, {s} с ΔQ={best_dQ:.6f}")

            # Объединение сообществ
            communities[r] = communities[r].union(communities[s])
            communities[s] = set()
            
            for node in communities[r]:
                comm_id[node] = r

            # Обновление a, e, Q
            a[r] = a[r] + a[s]
            del a[s]
            
            # Обновление e
            if s in e:
                for t in e[s]:
                    if t != r:
                        e[r][t] = e[r].get(t, 0) + e[s][t]
                        if t in e and s in e[t]:
                            e[t][r] = e[t].get(r, 0) + e[t][s]
                            del e[t][s]
                del e[s]
            
            if r in e and s in e[r]:
                del e[r][s]
            
            Q += best_dQ

        final_comms = [c for c in communities.values() if c]
        return final_comms, Q


In [ ]:
# Для ускорения работы используем подграф
sample_size = 5000
sample_nodes = list(G.nodes())[:sample_size]
G_test = G.subgraph(sample_nodes)

# Подготовка рёбер для CNM
edges = [(u, v, 1) for u, v in G_test.edges()]

print("Запуск CNM...")
model = CNM(edges)
comms, Q_my = model.fit(verbose=False)

# NetworkX эталон
print("Запуск NetworkX greedy_modularity_communities...")
nx_comms = list(greedy_modularity_communities(G_test))
Q_nx = modularity(G_test, nx_comms)

print("\n=== Сравнение ===")
print("CNM (наш)  Q =", round(Q_my, 5))
print("NetworkX    Q =", round(Q_nx, 5))
print("Количество сообществ (CNM):", len(comms))
print("Количество сообществ (NetworkX):", len(nx_comms))


Визуализация результатов

In [ ]:
plt.figure(figsize=(20, 10))
sub_nodes = list(G.nodes)[:1000]
H = G.subgraph(sub_nodes)

# создаём отображение: node -> community_id
node2comm = {}
for i, comm in enumerate(nx_comms):
    for n in comm:
        node2comm[n] = i

colors = [node2comm.get(n, 0) for n in H.nodes()]
pos = nx.spring_layout(H, k=0.3, iterations=30)
nx.draw(H, pos, node_color=colors, node_size=40, edge_color="gray", 
        with_labels=False, cmap="tab20", alpha=0.8)
plt.title("График окрашен по сообществам (NetworkX CNM)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()